In [1]:
%%time
import numpy as np 
import pandas as pd 

CPU times: user 508 ms, sys: 354 ms, total: 862 ms
Wall time: 1.81 s


In [2]:
%%time
#Loading the dataset
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
train_resource = pd.read_csv("input/resources.csv")

<string>:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 7.28 s, sys: 610 ms, total: 7.89 s
Wall time: 10.7 s


In [3]:
%%time
#Added a new column for total price per row
train_resource['value'] = train_resource['quantity'] * train_resource['price'] 

CPU times: user 65.9 ms, sys: 28.9 ms, total: 94.7 ms
Wall time: 227 ms


In [4]:
%%time
train_resource_grouped = train_resource.groupby(['id'], as_index=False)[['value']].sum()
test_resource_grouped = train_resource.groupby(['id'], as_index=False)[['value']].sum()

CPU times: user 1.27 s, sys: 79.5 ms, total: 1.35 s
Wall time: 1.35 s


In [5]:
%%time
#Train & Test dataset - resource part added
train_joined = pd.merge(train,train_resource_grouped,on='id')
test_joined = pd.merge(test,test_resource_grouped,on='id')

CPU times: user 495 ms, sys: 80.5 ms, total: 576 ms
Wall time: 574 ms


In [6]:
%%time
#Replacing NaN with empty string
#Addding essay 1 & 3
#Adding essay 2 & 4
#Effectively every row will have 2 essay columns
train_joined['projectessay_1_3'] = train_joined['project_essay_1']+train_joined['project_essay_3'].replace(np.nan, '', regex=True)
train_joined['projectessay_2_4'] = train_joined['project_essay_2']+train_joined['project_essay_4'].replace(np.nan, '', regex=True)
test_joined['projectessay_1_3'] = test_joined['project_essay_1']+test_joined['project_essay_3'].replace(np.nan, '', regex=True)
test_joined['projectessay_2_4'] = test_joined['project_essay_2']+test_joined['project_essay_4'].replace(np.nan, '', regex=True)

CPU times: user 112 ms, sys: 346 µs, total: 112 ms
Wall time: 110 ms


In [7]:
%%time
#Dropping id and previous essay columns
columns_to_remove = ["id","project_essay_1","project_essay_2","project_essay_3","project_essay_4"]
train_joined.drop(columns_to_remove,inplace=True,axis=1)
test_joined.drop(columns_to_remove,inplace=True,axis=1)

CPU times: user 265 ms, sys: 23.6 ms, total: 288 ms
Wall time: 288 ms


In [8]:
%%time
#Classifying acc to the type of cols
categorical_columns = ['teacher_prefix','school_state', 'project_grade_category',
                       'project_subject_categories', 'project_subject_subcategories']
non_cat_columns = ["project_submitted_datetime",
                   "teacher_number_of_previously_posted_projects","value"]
text_columns = ["project_title","project_resource_summary",
                "projectessay_1_3","projectessay_2_4"]

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 10 µs


In [9]:
%%time
# y (aka target var) is train['project_is_approved']
train_cat = train_joined[categorical_columns]
train_non_cat = train_joined[non_cat_columns]
train_text = train_joined[text_columns]
test_cat = test_joined[categorical_columns]
test_non_cat = test_joined[non_cat_columns]
test_text = test_joined[text_columns]
y = train_joined['project_is_approved']

CPU times: user 42.8 ms, sys: 0 ns, total: 42.8 ms
Wall time: 42 ms


In [10]:
%%time
train_cat = pd.get_dummies(train_cat)
test_cat = pd.get_dummies(test_cat)

CPU times: user 1.03 s, sys: 87.5 ms, total: 1.12 s
Wall time: 1.12 s


In [11]:
%%time
test_columns = test_cat.columns 
train_columns = train_cat.columns
all_columns = train_columns.union(test_columns)
train_add_columns = all_columns.difference(train_columns)
test_add_columns = all_columns.difference(test_columns)
test_copy = test_cat
test_cat = pd.concat([test_copy , test_copy.reindex(columns = test_add_columns, fill_value = 0.0)], axis = 1)
train_copy = train_cat
train_cat = pd.concat([train_copy , train_copy.reindex(columns = train_add_columns, fill_value = 0.0)], axis = 1)

CPU times: user 71.2 ms, sys: 164 ms, total: 235 ms
Wall time: 237 ms


In [12]:
from datetime import datetime

In [13]:
%%time
#Convert column to datetime column
train_non_cat["project_submitted_datetime"] = pd.to_datetime(train_non_cat["project_submitted_datetime"])
test_non_cat["project_submitted_datetime"] = pd.to_datetime(test_non_cat["project_submitted_datetime"])

CPU times: user 145 ms, sys: 3.51 ms, total: 148 ms
Wall time: 166 ms


/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
%%time
#Get month year from the datetome column
train_non_cat["Project_submitted_month"] = train_non_cat["project_submitted_datetime"].map(lambda x: x.month)
train_non_cat["Project_submitted_year"] = train_non_cat["project_submitted_datetime"].map(lambda x: x.year)
test_non_cat["Project_submitted_month"] = test_non_cat["project_submitted_datetime"].map(lambda x: x.month)
test_non_cat["Project_submitted_year"] = test_non_cat["project_submitted_datetime"].map(lambda x: x.year)

/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 1.77 s, sys: 16.3 ms, total: 1.78 s
Wall time: 1.79 s


/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [15]:
%%time
train_non_cat.drop(["project_submitted_datetime"],inplace=True,axis=1)
test_non_cat.drop(["project_submitted_datetime"],inplace=True,axis=1)

CPU times: user 83.8 ms, sys: 0 ns, total: 83.8 ms
Wall time: 82 ms


/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/sannisth/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
%%time
std = StandardScaler()

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 13.4 µs


In [18]:
%%time
train_non_cat_scaled = pd.DataFrame(std.fit_transform(train_non_cat),columns=train_non_cat.columns)
test_non_cat_scaled = pd.DataFrame(std.fit_transform(test_non_cat),columns=test_non_cat.columns)

CPU times: user 26.5 ms, sys: 276 µs, total: 26.8 ms
Wall time: 26.5 ms


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix

In [20]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    lowercase=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 70.1 µs


In [21]:
%%time
vectorized_columns = []
vectorized_columns_test = []
for text_column in text_columns:
    word_vectorizer.fit(pd.concat([train_text[text_column],test_text[text_column]]))
    vectorized_columns.append(word_vectorizer.transform(train_text[text_column]))
    vectorized_columns_test.append(word_vectorizer.transform(test_text[text_column]))


CPU times: user 2min 4s, sys: 1.08 s, total: 2min 5s
Wall time: 2min 5s


In [22]:
%%time
train_features = hstack(vectorized_columns+[csr_matrix(train_non_cat_scaled)]+[csr_matrix(train_cat)], 'csr')
test_features = hstack(vectorized_columns_test+[csr_matrix(test_non_cat_scaled)]+[csr_matrix(test_cat)], 'csr')

CPU times: user 4.16 s, sys: 868 ms, total: 5.03 s
Wall time: 5.05 s


In [34]:
%%time
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc.fit(train_features,y)

predicted_prolly_rfc = rfc.predict_proba(test_features)
result_rfc = pd.DataFrame.from_dict({'id':test['id']})
result_rfc['project_is_approved'] = predicted_prolly_rfc[:,1]
result_rfc.to_csv("submission_rfc.csv",index=False)

CPU times: user 2.52 s, sys: 248 ms, total: 2.77 s
Wall time: 3.42 s


In [35]:
%%time
from sklearn.linear_model import SGDClassifier
sgdclassifier = SGDClassifier(loss="log",penalty="l2")
sgdclassifier.fit(train_features,y)
predicted_prolly = sgdclassifier.predict_proba(test_features)
#result = pd.DataFrame.from_dict({'id':test['id']})
#result['project_is_approved'] =

/home/sannisth/venv/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


CPU times: user 1.34 s, sys: 11.7 ms, total: 1.36 s
Wall time: 2.15 s


In [36]:
%%time
result = pd.DataFrame.from_dict({'id':test['id']})
result['project_is_approved'] = predicted_prolly[:,1]
result.to_csv("submission_sgd.csv",index=False)

CPU times: user 243 ms, sys: 76 µs, total: 243 ms
Wall time: 245 ms


In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
%%time
model = LogisticRegression(n_jobs=-1)
model.fit(train_features,y)

/home/sannisth/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 1min 2s, sys: 92.2 ms, total: 1min 2s
Wall time: 1min 2s


In [39]:
%%time
predicted_proba = model.predict_proba(test_features)
submission = pd.DataFrame.from_dict({'id': test['id']})
submission['project_is_approved'] = predicted_proba[:,1]
submission.to_csv("submission.csv",index=False)

CPU times: user 284 ms, sys: 159 µs, total: 285 ms
Wall time: 285 ms


In [40]:
%%time
#Trying LightGBM
print("Building model using Light GBM and finding AUC(Area Under Curve)")
from sklearn.model_selection import train_test_split
X_train_l, X_valid_l, y_train_l, y_valid_l = train_test_split(train_features, y, test_size=0.10, random_state=2018)
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 7,
        'num_leaves': 32,
        'learning_rate': 0.02,
        'feature_fraction': 0.80,
        'bagging_fraction': 0.80,
        'bagging_freq': 5,
        'verbose': 0,
        'lambda_l2': 1,
    }  

import lightgbm as lgb
evals_result = {}  # to record eval results for plotting
model_lgb = lgb.train(
        params,
        lgb.Dataset(X_train_l, y_train_l),
        num_boost_round=10000,
        valid_sets=[lgb.Dataset(X_valid_l, y_valid_l)],
        early_stopping_rounds=100,
        evals_result=evals_result,
        verbose_eval=25)
from sklearn.metrics import roc_auc_score
valid_preds_lgb = model_lgb.predict(X_valid_l, num_iteration=model_lgb.best_iteration)
test_preds = model_lgb.predict(test_features, num_iteration=model_lgb.best_iteration)
auc = roc_auc_score(y_valid_l, valid_preds_lgb)
print('AUC:',auc)

Building model using Light GBM and finding AUC(Area Under Curve)
Training until validation scores don't improve for 100 rounds.
[25]	valid_0's auc: 0.678753
[50]	valid_0's auc: 0.688007
[75]	valid_0's auc: 0.692977
[100]	valid_0's auc: 0.702708
[125]	valid_0's auc: 0.710777
[150]	valid_0's auc: 0.71666
[175]	valid_0's auc: 0.72158
[200]	valid_0's auc: 0.726588
[225]	valid_0's auc: 0.730336
[250]	valid_0's auc: 0.734045
[275]	valid_0's auc: 0.736989
[300]	valid_0's auc: 0.739186
[325]	valid_0's auc: 0.741367
[350]	valid_0's auc: 0.743142
[375]	valid_0's auc: 0.744646
[400]	valid_0's auc: 0.745842
[425]	valid_0's auc: 0.747229
[450]	valid_0's auc: 0.748748
[475]	valid_0's auc: 0.750011
[500]	valid_0's auc: 0.75085
[525]	valid_0's auc: 0.75169
[550]	valid_0's auc: 0.752704
[575]	valid_0's auc: 0.753547
[600]	valid_0's auc: 0.753945
[625]	valid_0's auc: 0.754923
[650]	valid_0's auc: 0.755393
[675]	valid_0's auc: 0.756066
[700]	valid_0's auc: 0.756832
[725]	valid_0's auc: 0.757521
[750]	val

In [41]:
%%time
submission1 = pd.DataFrame.from_dict({'id': test['id']})
submission1['project_is_approved'] = test_preds
submission1.to_csv("submission1.csv",index=False)

CPU times: user 247 ms, sys: 7.97 ms, total: 255 ms
Wall time: 609 ms


In [43]:
%%time
#Trying XGBoost
from sklearn.model_selection import train_test_split
import xgboost as xgb
xgb_params = {'eta': 0.2, 
                  'max_depth': 5, 
                  'subsample': 0.8, 
                  'colsample_bytree': 0.8, 
                  'objective': 'binary:logistic', 
                  'eval_metric': 'auc', 
                  'seed': 1234
                 }
X_train_x, X_valid_x, y_train_x, y_valid_x = train_test_split(train_features, y, test_size=0.33, random_state=2018)
d_train = xgb.DMatrix(X_train_x, y_train_x)
d_valid = xgb.DMatrix(X_valid_x, y_valid_x)
d_test = xgb.DMatrix(test_features)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model_xgb = xgb.train(xgb_params, d_train, 500, watchlist, verbose_eval=50, early_stopping_rounds=100)
xgb_pred_test = model_xgb.predict(d_test)
xgb_pred_valid = model_xgb.predict(d_valid)
auc = roc_auc_score(y_valid_x, xgb_pred_valid)
print('AUC:',auc)

[0]	train-auc:0.611914	valid-auc:0.609135
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 100 rounds.
[50]	train-auc:0.78454	valid-auc:0.727543
[100]	train-auc:0.823112	valid-auc:0.737252
[150]	train-auc:0.848884	valid-auc:0.740282
[200]	train-auc:0.870085	valid-auc:0.742116
[250]	train-auc:0.886647	valid-auc:0.742301
[300]	train-auc:0.900448	valid-auc:0.743686
[350]	train-auc:0.911879	valid-auc:0.744054
[400]	train-auc:0.922382	valid-auc:0.744138
Stopping. Best iteration:
[335]	train-auc:0.908244	valid-auc:0.744598

AUC: 0.7438332337078135
CPU times: user 38min 44s, sys: 4.07 s, total: 38min 48s
Wall time: 10min 43s


In [32]:
# xgb_pred_test

In [33]:
%%time
submission2 = pd.DataFrame.from_dict({'id': test['id']})
submission2['project_is_approved'] = xgb_pred_test
submission2.to_csv("submission2.csv",index=False)